In [12]:
import pandas as pd
predicted_country_df = pd.read_csv("predicted_temp_country.csv")
predicted_country_df

,Country,Year,avg_temp_F
0,Afghanistan,2013,59.620016
1,Afghanistan,2014,59.669303
2,Afghanistan,2015,59.718589
3,Afghanistan,2016,59.767875
4,Afghanistan,2017,59.817161
...,...,...,...
19267,Zimbabwe,2096,73.153647
19268,Zimbabwe,2097,73.175849
19269,Zimbabwe,2098,73.198050
19270,Zimbabwe,2099,73.220251


In [14]:
previous_data = pd.read_csv("country_temp_data_ML.csv")
previous_data = previous_data[["Country","Year","avg_temp_F"]]
full_country_data = pd.concat([predicted_country_df,previous_data],ignore_index=True)
full_country_data = full_country_data.sort_values(["Country","Year"],ascending = True).reset_index(drop=True)
full_country_data

,Country,Year,avg_temp_F
0,Afghanistan,1900,56.748800
1,Afghanistan,1901,57.009200
2,Afghanistan,1902,58.110500
3,Afghanistan,1903,55.375550
4,Afghanistan,1904,56.850350
...,...,...,...
44014,Zimbabwe,2096,73.153647
44015,Zimbabwe,2097,73.175849
44016,Zimbabwe,2098,73.198050
44017,Zimbabwe,2099,73.220251


In [15]:
import os
import shutil
import time

import folium
import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

In [17]:
beg_temp = 0
full_country_data["year change"] = 0
full_country_data["running_total_change"] = 0

for index, row in full_country_data.iterrows():
    if row["Year"] == 1900:
        full_country_data.loc[index, 'year change'] = 0
        full_country_data.loc[index, 'running_total_change'] = 0
        beg_temp = row["avg_temp_F"]
        running_total = 0
    else:
        change = row["avg_temp_F"] - beg_temp
        running_total = running_total + change
        full_country_data.loc[index, 'year change'] = change
        full_country_data.loc[index, 'running_total_change'] = running_total
        beg_temp = row["avg_temp_F"]
        
full_country_data.head(50)

,Country,Year,avg_temp_F,year change,running_total_change
0,Afghanistan,1900,56.74880,0.00000,0.00000
1,Afghanistan,1901,57.00920,0.26040,0.26040
2,Afghanistan,1902,58.11050,1.10130,1.36170
3,Afghanistan,1903,55.37555,-2.73495,-1.37325
4,Afghanistan,1904,56.85035,1.47480,0.10155
5,Afghanistan,1905,56.09420,-0.75615,-0.65460
6,Afghanistan,1906,56.99285,0.89865,0.24405
7,Afghanistan,1907,56.03525,-0.95760,-0.71355
8,Afghanistan,1908,56.65850,0.62325,-0.09030
9,Afghanistan,1909,57.06575,0.40725,0.31695


In [23]:
for i in range(2013,2101):
    data = full_country_data.loc[full_country_data["Year"]== i]
    data.loc[data['running_total_change']<0,"running_total_change"] = 0
    


    bins = [0,1,2,3,4,10]

    m_i = folium.Map(location=[0,0], width=1100, height=600,zoom_start=1.5)
    
    title_html = '''
                 <h3 align="left" style="font-size:22px"><b>{}</b></h3>
                 '''.format('Year: ' + str(i))   
    m_i.get_root().html.add_child(folium.Element(title_html))

    folium.Choropleth(
        geo_data="datasets/custom.geo.json",
        name='choropleth',
        data=data,
        columns=['Country', 'running_total_change'],
        key_on='feature.properties.sovereignt',
        fill_color='OrRd',
        fill_opacity=0.9,
        line_opacity=0.2,
        legend_name='Degree Change',
        bins=bins
    ).add_to(m_i)

    folium.LayerControl().add_to(m_i)
    m_i.save('datasets/map_'+str(i)+'.html')

/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is tryin

/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is tryin

/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is tryin

/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is tryin

/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/robertsimeonoglou/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is tryin